In [ ]:
import pandas as pd
import folium
from math import radians, cos, sin, asin, sqrt
from geopy.geocoders import Nominatim
from IPython.display import display

# ============================
# 📁 데이터 로딩
# ============================
summary_df = pd.read_csv("watcha_drama_summary_all.csv")
place_reviews_df = pd.read_csv("drama_locations_for_map.csv")
drama_reviews_df = pd.read_csv("drama_main_cluster_labeled.csv")

# ============================
# 🎭 감정 선택
# ============================
labels = sorted(drama_reviews_df["label"].unique().tolist())
print("🎭 원하는 감정 상태를 선택하세요:")
for i, label in enumerate(labels):
    print(f"{i+1}. 클러스터 {label}")

choice = int(input("번호 입력: ")) - 1
selected_label = labels[choice]
print(f"\n📌 선택한 감정 클러스터: {selected_label}")

# ============================
# 🎬 드라마 추천
# ============================
selected_dramas = drama_reviews_df[drama_reviews_df["label"] == selected_label]
print("\n🎬 추천 드라마 목록:")
for i, row in enumerate(selected_dramas.itertuples()):
    summary = summary_df[summary_df["title"] == row.drama]["summary"]
    summary_text = summary.values[0] if not summary.empty else "한줄 요약 없음"
    print(f"{i+1}. {row.drama} - {summary_text}")

# ============================
# 📍 드라마 선택
# ============================
drama_idx = int(input("\n보고 싶은 드라마 번호를 선택하세요: ")) - 1
drama_name = selected_dramas.iloc[drama_idx]["drama"]
print(f"\n📍 선택한 드라마: {drama_name}")

# ============================
# 📌 촬영지 정제 및 그룹화
# ============================
def clean_text(text):
    return str(text).strip()

drama_places = place_reviews_df[place_reviews_df["mediatitle"] == drama_name].copy()

# 장소명 + 주소 정제
drama_places["place_name_clean"] = drama_places["place_name"].apply(clean_text)
drama_places["address_clean"] = drama_places["address"].apply(clean_text)
drama_places["dedup_key"] = drama_places["place_name_clean"] + "_" + drama_places["address_clean"]

# dedup_key 기준 중복 제거
drama_places = drama_places.sort_values(by="positive_ratio", ascending=False)
drama_places = drama_places.drop_duplicates(subset="dedup_key")

# 그룹화
grouped_places = (
    drama_places
    .groupby(["place_name", "address", "latitude", "longitude"], as_index=False)
    .agg({"positive_ratio": "max"})
)

grouped_places = grouped_places.sort_values(by="positive_ratio", ascending=False)

if grouped_places.empty:
    print("😢 해당 드라마의 촬영지 정보가 없습니다.")
    exit()

# ============================
# 📍 주소 → 좌표 변환 함수
# ============================
def address_to_coordinates(address):
    geolocator = Nominatim(user_agent="drama_location_recommender")
    location = geolocator.geocode(address)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

# ============================
# 🧭 사용자 주소 입력 및 변환
# ============================
user_address = input("\n📍 당신의 현재 위치 주소를 입력하세요 (예: 서울특별시 중구 세종대로 110): ")
user_lat, user_lon = address_to_coordinates(user_address)

if user_lat is None or user_lon is None:
    print("❌ 주소를 찾을 수 없습니다. 다시 시도해주세요.")
    exit()

print(f"🗺️ 변환된 좌표: 위도 {user_lat}, 경도 {user_lon}")

# ============================
# 📏 거리 계산 함수
# ============================
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # 지구 반지름 (km)
    return c * r

# ============================
# 📌 거리 계산
# ============================
grouped_places["distance_from_user_km"] = grouped_places.apply(
    lambda row: haversine(user_lat, user_lon, row["latitude"], row["longitude"]), axis=1
)

# ============================
# 📌 거리 + 감정 몰입 순 출력 (📌 중복 place_name은 1번만 출력)
# ============================
print(f"\n💖 감정 몰입도 높은 촬영지 거리 목록:\n")

printed_places = set()  # 이미 출력한 장소명 저장

for row in grouped_places.itertuples():
    if row.place_name in printed_places:
        continue
    printed_places.add(row.place_name)

    print(f"{len(printed_places)}. {row.place_name} ({row.address})")
    print(f"   👉 감정 긍정 비율: {round(row.positive_ratio * 100)}%, 거리: {round(row.distance_from_user_km, 2)}km\n")

# ============================
# 🗺️ 지도 시각화
# ============================
print("🗺️ 촬영지와 사용자 위치를 지도에 표시합니다.")

m = folium.Map(location=[user_lat, user_lon], zoom_start=11, tiles="CartoDB positron")

# 사용자 위치 마커
folium.Marker(
    location=[user_lat, user_lon],
    popup="📍 나의 위치",
    icon=folium.Icon(color="blue", icon="user", prefix="fa")
).add_to(m)

# 촬영지 마커 (중복 place_name 관계없이 전체 표시)
for _, row in grouped_places.iterrows():
    if pd.notnull(row["latitude"]) and pd.notnull(row["longitude"]):
        folium.Marker(
            location=[row["latitude"], row["longitude"]],
            popup=folium.Popup(
                f"<b>{row['place_name']}</b><br>{row['address']}<br>긍정 비율: {round(row['positive_ratio'] * 100)}%<br>거리: {round(row['distance_from_user_km'], 2)}km",
                max_width=300
            ),
            icon=folium.Icon(color="green", icon="film", prefix="fa")
        ).add_to(m)

display(m)

